In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from gears import PertData, GEARS
import numpy as np
import pandas as pd
from tqdm import tqdm
import scanpy as sc
import anndata
from scipy.sparse import csr_matrix

In [5]:
data_dir = '/mlbio_scratch/vinas/sc_perturbation_benchmark/data/replogle_rpe1_v2_2022'

#### Download data

Data access: https://gwps.wi.mit.edu/

* Files:
    * rpe1_raw_singlecell_01.h5ad (https://plus.figshare.com/articles/dataset/_Mapping_information-rich_genotype-phenotype_landscapes_with_genome-scale_Perturb-seq_Replogle_et_al_2022_processed_Perturb-seq_datasets/20029387)
    * annotated_embedding_coordinates.csv (https://plus.figshare.com/articles/dataset/_Mapping_information-rich_genotype-phenotype_landscapes_with_genome-scale_Perturb-seq_Replogle_et_al_2022_-_commonly_requested_supplemental_files/21632564/1)

In [7]:
adata = anndata.read_h5ad(f'{data_dir}/rpe1_raw_singlecell_01.h5ad')
df_sub = pd.read_csv(f'{data_dir}/annotated_embedding_coordinates.csv')

#### scGPT processing
1. Select genes in annotated embedding coordinates, corresponding to perturbations that elicit strong phenotype changes.
2. Downsample 100 samples per perturbation and 2500 control samples
3. Normalize data (unsure why the scGPT authors performed HVG selection after downsampling).
4. Keep HVGs and genes that were perturbed

In [11]:
adata_ = adata[adata.obs['gene'].isin(list(df_sub.gene.values)+['non-targeting']), :]

In [14]:
# Subsample to 100 samples per perturbation and 2500 control samples
target_cells = 100
cluster_key = 'gene'
adatas = [adata_[adata_.obs[cluster_key]==clust] for clust in adata_.obs[cluster_key].cat.categories]
for dat in tqdm(adatas):
    if dat.n_obs > target_cells:
        if dat.obs[cluster_key].cat.categories.values[0] != 'non-targeting':
            sc.pp.subsample(dat, n_obs=target_cells)
        else:
            sc.pp.subsample(dat, n_obs=2500)
adata_downsampled_ = adatas[0].concatenate(*adatas[1:])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1527/1527 [00:03<00:00, 464.40it/s]


In [15]:
sc.pp.normalize_total(adata_downsampled_)
sc.pp.log1p(adata_downsampled_)
sc.pp.highly_variable_genes(adata_downsampled_, n_top_genes=5000, subset=False)
hvg_flag_ = adata_downsampled_.var['highly_variable'].values
gene_flag_ = adata_downsampled_.var['gene_name'].isin(adata_downsampled_.obs['gene'].values).values
select_flag_ = np.logical_or(hvg_flag_, gene_flag_)
condition_flag_ = adata_downsampled_.obs['gene'].isin(adata_downsampled_.var['gene_name'].values.tolist()+['non-targeting']).values
adata_subset = adata_downsampled_[condition_flag_, select_flag_]

In [16]:
# Prepare data in GEARS format
adata_subset.obs['condition'] = [i+'+ctrl' for i in adata_subset.obs['gene'].values]
adata_subset.obs['condition'] = adata_subset.obs['condition'].replace({'non-targeting+ctrl': 'ctrl'})
adata_subset.obs['cell_type'] = 'RPE1'
len(adata_subset.obs['condition'].unique())

1416

In [17]:
out_dir = '/mlbio_scratch/vinas/sc_perturbation_benchmark/data/replogle_rpe1_v2_2022'
pert_data = PertData(out_dir)
adata_subset.X = csr_matrix(adata_subset.X)
pert_data.new_data_process(dataset_name = 'rpe1_1900_{}'.format(target_cells), adata = adata_subset)

Downloading...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:01<00:00, 8.13MiB/s]
Downloading...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 559k/559k [00:00<00:00, 1.15MiB/s]
Creating pyg object for each cell in the data...
Creating dataset file...
  1%|█▋                                                                                                                                                                                                                                                                         | 9/1416 [00

AC118549.1+ctrl


  6%|████████████████▋                                                                                                                                                                                                                                                         | 89/1416 [00:32<04:20,  5.09it/s]

C7orf26+ctrl


  6%|█████████████████▎                                                                                                                                                                                                                                                        | 92/1416 [00:33<04:41,  4.70it/s]

...

 26%|████████████████████████████████████████████████████████████████████▍                                                                                                                                                                                                    | 366/1416 [02:25<06:47,  2.58it/s]

FAU+ctrl


 77%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                            | 1092/1416 [06:43<01:42,  3.17it/s]

SEM1+ctrl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1416/1416 [08:48<00:00,  2.68it/s]
Done!
Saving new dataset pyg object at /mlbio_scratch/vinas/sc_perturbation_benchmark/data/replogle_rpe1_v2_2022/rpe1_1900_100/data_pyg/cell_graphs.pkl
Done!
